In [6]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
beers = pd.read_csv('beers-breweries-and-beer-reviews/beers.csv')
breweries = pd.read_csv('beers-breweries-and-beer-reviews/breweries.csv')
reviews = pd.read_csv('beers-breweries-and-beer-reviews/reviews.csv')

In [4]:
reviews['text'] = reviews['text'].replace(u'\xa0\xa0', '')
# subset to only reviews that have a text review
text_reviews = reviews.loc[reviews['text'] != '']
# subset data to exclude NaN's as well (only losing 164k reviews from the last subset)
text_no_nan = text_reviews.loc[text_reviews.smell.isna() == False]
# rename column name beer_id to id for easy joining
text_no_nan = text_no_nan.rename(columns={'beer_id':'id'})
# subset out retired beers
current_beers = beers.loc[beers['retired'] == 'f']
# merge text_no_nan with beers that are not retired
df = pd.merge(text_no_nan, current_beers, on='id')
# create a table with average ratings for each beer. Index/ID is the beer id
ratings = pd.DataFrame(df.groupby('id')['score'].mean())
# add a column tallying the # of reviews for that beer
ratings['no_of_ratings'] = df.groupby('id')['score'].count()
# subset ratings with only beers that have 10+ ratings
ratings = ratings.loc[ratings['no_of_ratings'] > 9]
# formatting
ratings = ratings.reset_index()
ratings = ratings.rename(columns={'score':'avg_score'})
# rejoin no of ratings onto df
df = df.merge(ratings, how='inner', on='id')
# make a dataframe of reviewers by usename, count the number of reviews they made
reviewers = pd.DataFrame(df.groupby('username')['id'].count())
# make a new feature, the average of all of their scores
reviewers['avg_usr_score'] = df.groupby('username')['score'].mean()
# subset reviewers to those with 5+ reviews. From 73k users to 25k.
reviewers = reviewers.loc[reviewers['id'] > 4] ## MAYBE I CAN PLAY WITH THIS #
# formatting 
reviewers = reviewers.rename(columns={'id':'tot_usr_rvw'})
# there's ~1400 users outsides of 2 STDs of the mean score, will subset them out
reviewers_sub = reviewers.loc[(reviewers['avg_usr_score'] >= 3.182) &\
                              (reviewers['avg_usr_score'] <= 4.665)]

# subset of df with beers that have 10+ reviews, and with reviewers that have 5+ reviews
# and an average rating of beers between 3.18 and 4.67
df_with_mins = df.merge(reviewers_sub, how = 'inner', on = 'username')

Content Based NLP Stuff:

In [47]:
# This is the combined text for each beer. this skips most preprocessing below. 
df_joined = pd.read_pickle('joined_text_df.pkl')

In [48]:
lookup_df = df_joined[['id', 'brewery_id', 'city', 'state', 'country', 'brewery_name']]
lookup_dict = lookup_df.set_index('id').to_dict(orient='index')

In [129]:
# this is each review cleaned for Doc2Vec
cleaned_reviews_df = pd.read_pickle('cleaned_reviews_df.pkl')

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from sklearn.feature_extraction import text
from gensim.parsing.preprocessing import STOPWORDS as stop_words

In [4]:
# subsets reviews df, and then joins all text reviews for each individual beer together
df_joined = df_with_mins.copy()
df_joined['joined_text'] = df_joined.groupby('id')['text'].\
                               transform(lambda x: ''.join(x))

In [5]:
# removes duplicate beers, and subsets to just beer id, joined_text and rating info
# to be cleaned and then joined to beers df
df_joined_sub = df_joined[['id', 'joined_text', 'avg_score', 'no_of_ratings']].drop_duplicates(\
                                                                        subset='id')

In [6]:
# removes \xa0 remove text
df_joined_sub['joined_text'] = df_joined_sub['joined_text'].apply(lambda x: re.sub\
                                                                  (r'\xa0', '', x))

In [19]:
breweries_sub = breweries[['id','name', 'city', 'state', 'country']]
breweries_sub = breweries_sub.rename(columns={'id':'brewery_id', 'name':'brewery_name'})

In [20]:
beers_sub = beers[['id', 'name', 'brewery_id', 'style', 'abv']]

In [21]:
breweries_beer = pd.merge(beers_sub, breweries_sub, on = 'brewery_id')

In [26]:
joined_df = pd.merge(df_joined_sub, breweries_beer, on = 'id')

In [28]:
joined_df.to_pickle("./joined_text_df.pkl")

In [14]:
# make a separate df with each individual review, clean up \xa0's and then pickle
cleaned_reviews = df_with_mins[['id', 'text']]
cleaned_reviews['text'] = cleaned_reviews['text'].apply(lambda x: re.sub(r'\xa0', '', x))
# cleaned_reviews.to_pickle("./cleaned_reviews_df.pkl")

//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
cleaned_reviews.text[0]

' 0% 16 oz can. Funny story: As I finally walked in the doors after a 45 min wait in line and freezing temps the sweet sound of the Grateful Dead\'s Sugar Magnolia greeted me from the TreeHouse sound system. The bottom of the can reads: "Going where the wind goes, bloomin\' like a red rose" A white haze to the yellow and golden liquid. Thick and healthy, totally unfiltered. Brawny white foam cap, thick, all-white clumps. Huge lacing left over. The aroma has a very zesty citrus hop effect, mellon and mango, grainy earthiness, tropical fruit blend with a bitter to sweet effect, then a peppery kick at the end. Very aromatic. The flavor is just bursting with complex hops, zesty earthy tones, sweet orange, peppery malt, clean fresh feel and overall vibe. A crispy bite wakes you up, full and lush mouthfeel follows from a totally unfiltered expereince. The feel and flavor finishes with a fun, earthy, zesty dry bite. Tropical juicy, zesty citrus, zippy golden wheat malt, melons, rustic earthin

In [6]:
count_vect = CountVectorizer(stop_words='english')
counts = count_vect.fit_transform(df_joined_sub.joined_text)
cos_sim = cosine_similarity(counts, counts)

KeyboardInterrupt: 

In [0]:
indices = pd.Series(df_joined_sub.index)

In [29]:
def recommendations(beer_id, df, cos_sim = cos_sim):
    """
    Takes a beer id and cosine similarty matrix in as arguments and returns beers closely related to the input beer
    """
    # initializing the empty list of recommended movies
    recommended_beers = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == beer_id].index[0]
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_beers.append(list(joined_df.name)[i])
        
    return recommended_beers

def tfidf_recs(beer_id, cos_sim = tfidf_cos):
    """
    Takes a beer id and cosine similarty matrix in as arguments and returns beers closely related to the input beer
    """
    # initializing the empty list of recommended movies
    recommended_beers = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == beer_id].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_beers.append(list(beers_text.name)[i])
        
    return beers_text.name[beer_id], recommended_beers

NameError: name 'cos_sim' is not defined

In [32]:
joined_df.sample()

,id,joined_text,avg_score,no_of_ratings,name,brewery_id,style,abv,brewery_name,city,state,country
16302,65010,Consumed 4/13/12 Appearance: Dark caramel/lig...,3.607619,42,Perfect Tin Amber,24488,American Amber / Red Ale,4.5,Tin Roof Brewing Company,Baton Rouge,LA,US


In [172]:
import string
import re

RE_PUNCT = re.compile('([%s])+' % re.escape(string.punctuation), re.UNICODE)

def preprocess(text):
    ls = LancasterStemmer()
    # Remove all punctuation and make all lowercase 
    return ls.stem(RE_PUNCT.sub(" ", text)).lower().split()

### DOC2VEC

In [1]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec, Phrases
from gensim.parsing.preprocessing import STOPWORDS as stop_words
from gensim.utils import simple_preprocess
from sklearn.feature_extraction import text
from nltk.stem.lancaster import LancasterStemmer

In [2]:
letters = list('abcdefghijklmnopqrstuvwxyz')
numbers = list('0123456789')
words = ['oz', 'ml'] # ADD MORE
stop_words = stop_words.union(set(letters)).union(set(numbers)).union(set(words))
my_stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)

In [72]:
# use Phrases to let the model detect bigrams
# bigram = Phrases(map(preprocess, df_sub.text.tolist()),max_vocab_size=10000000)


In [3]:
def preprocessor(text):
    # uses gensim simple_preprocess and then removes stop words
    simple = simple_preprocess(text)
    result = [word for word in simple if not word in my_stop_words]
    return result

In [4]:
# this takes the document and 'text' as arguments
# makes words lowercase and splits them, and then adds the beer id as tag
# returns as TaggedDocument
def tag_docs(docs):
    results = docs.apply(lambda r: TaggedDocument(words=preprocessor(r['text']), tags=[r['id']]), axis=1)
    return results.tolist()

In [249]:
# subsetting for a test
df_sub = cleaned_reviews_df[:10]

In [8]:
# process reviews, turn into list of TaggedDocument objects, with beer id as tag
tagged_docs = tag_docs(cleaned_reviews_df)

In [9]:
## Use format like this 

model = Doc2Vec(dm=0, dbow_words=1, min_count=4, negative=3,
                hs=0, sample=1e-4, window=5, size=100, workers=8)

model.build_vocab(tagged_docs, progress_per = 100)

//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [16]:
from gensim.models.callbacks import CallbackAny2Vec
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [17]:
epoch_logger = EpochLogger()
model.train(tagged_docs, total_examples=model.corpus_count, epochs=6, callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end


In [138]:
model.similar_by_vector('effervescence')

//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('effervescent', 0.8132021427154541),
 ('lively', 0.8083217144012451),
 ('fizziness', 0.7874601483345032),
 ('carbonation', 0.7864266037940979),
 ('effervesence', 0.7799913883209229),
 ('active', 0.777531087398529),
 ('sparkly', 0.7369555234909058),
 ('spritzy', 0.731114387512207),
 ('bubbliness', 0.726111114025116),
 ('liveliness', 0.7169278264045715)]

In [22]:
model.save('d2v-6epoch.pkl')

In [23]:
model.save_word2vec_format('d2v-format', doctag_vec= True, )

In [140]:
# this is the above but trying it to return a dict
def location_filter(ranked_beers, state, city, n):

    located_brewery = {}
    # state = 'CA'
    # city = 'Los Angeles'
    counter = 0

    for beer in ranked_beers:
        if counter < n:
            dict_state = lookup_dict[beer[0]]['state']
            dict_city = lookup_dict[beer[0]]['city']
            brewery_id = lookup_dict[beer[0]]['brewery_id']
            brewery_name = lookup_dict[beer[0]]['brewery_name']
            if (dict_state == state) and (dict_city == city):
        #             print(beer_breweries_lookup[beer[0]])
                print(beer[0])
                if brewery_id in located_brewery:
                    continue
                else:  
                    located_brewery[brewery_id] = brewery_name
                counter += 1
    return located_brewery

In [141]:
vec = model['hop']
d2v_test = model.docvecs.most_similar([vec], topn=3000)
# d2v_test
location_filter(d2v_test, 'WA', 'Bellingham', 2)


52038


KeyError: 284513

In [125]:
lookup_dict[284513]

KeyError: 284513

In [121]:
for doc in tagged_docs:
    print(doc.tags)

[125646]
[47678]
[71930]
[326798]
[48824]
[317815]
[253734]
[272971]
[52204]
[58999]
[61577]
[67760]
[69522]
[252546]
[163693]
[122334]
[255975]
[178857]
[135684]
[159480]
[286228]
[80702]
[350683]
[291168]
[250406]
[272639]
[295109]
[117177]
[273145]
[173856]
[87869]
[88361]
[170814]
[88969]
[260710]
[246247]
[234127]
[273571]
[265678]
[276812]
[255246]
[159676]
[99000]
[102133]
[104124]
[330409]
[238449]
[262713]
[127793]
[320520]
[125926]
[291222]
[205199]
[209637]
[262312]
[55]
[1826]
[34094]
[109105]
[40678]
[38224]
[104431]
[48194]
[817]
[169936]
[5328]
[2255]
[22555]
[288330]
[86237]
[305445]
[74]
[58261]
[149964]
[86241]
[253607]
[53453]
[124602]
[1493]
[112128]
[6]
[7]
[17]
[165750]
[2264]
[30]
[31]
[33]
[34]
[48541]
[36]
[39]
[63]
[20897]
[59]
[296]
[61]
[6861]
[65]
[73]
[69]
[16403]
[80]
[81]
[4307]
[83]
[85]
[88]
[93646]
[90]
[89]
[92]
[9745]
[99]
[26072]
[100]
[101]
[102]
[832]
[5653]
[103]
[45485]
[104]
[1608]
[108]
[147351]
[129]
[131]
[150]
[133]
[137]
[139]
[141]
[142]

[145496]
[262688]
[88298]
[210034]
[125646]
[58610]
[127574]
[48824]
[48873]
[49070]
[49374]
[55564]
[55939]
[65113]
[67046]
[123009]
[74986]
[151054]
[76816]
[178857]
[317146]
[162954]
[232399]
[79424]
[141390]
[83473]
[217619]
[100217]
[88854]
[139129]
[92697]
[155437]
[97840]
[199756]
[219910]
[245957]
[178740]
[108525]
[114300]
[199521]
[166562]
[181993]
[291222]
[262118]
[205199]
[139172]
[312418]
[121106]
[195]
[149097]
[37113]
[2255]
[78927]
[87556]
[72834]
[77627]
[74]
[79669]
[81986]
[1493]
[5]
[7]
[2264]
[23720]
[34]
[247502]
[63]
[65]
[73]
[27800]
[38394]
[78]
[16403]
[123417]
[88]
[92]
[102]
[5653]
[103]
[104]
[108]
[132]
[141]
[66752]
[122678]
[148]
[155]
[159]
[1173]
[179482]
[198780]
[197]
[198]
[2178]
[219]
[85178]
[224]
[449]
[232]
[77482]
[233]
[124710]
[248]
[933]
[93373]
[276]
[39435]
[162643]
[34420]
[299]
[2184]
[310]
[311]
[58048]
[52077]
[332]
[345]
[299015]
[355]
[293771]
[7463]
[244539]
[412]
[262827]
[144097]
[113602]
[129357]
[62645]
[57487]
[916]
[40695]
[5

[99989]
[149592]
[95445]
[48254]
[112667]
[170112]
[44932]
[30764]
[148052]
[60371]
[117409]
[4810]
[31548]
[6076]
[197751]
[28165]
[208671]
[20768]
[97432]
[87118]
[90671]
[145763]
[54904]
[23459]
[5133]
[54657]
[253963]
[152165]
[27804]
[141397]
[150766]
[5577]
[109010]
[7077]
[116721]
[6227]
[21120]
[26792]
[197086]
[94191]
[200821]
[59332]
[48911]
[57210]
[130699]
[20518]
[6225]
[90585]
[140014]
[9612]
[2269]
[88478]
[83599]
[20205]
[212266]
[76086]
[249893]
[84565]
[96677]
[46366]
[78501]
[84747]
[142442]
[28039]
[3158]
[85847]
[93245]
[253068]
[56414]
[100443]
[66594]
[88341]
[13889]
[223064]
[35036]
[86172]
[43181]
[51390]
[201499]
[2477]
[91559]
[169625]
[48243]
[25009]
[153763]
[143754]
[227005]
[107907]
[23154]
[144870]
[33183]
[86500]
[43421]
[69702]
[110462]
[174714]
[41815]
[96558]
[19922]
[82605]
[32413]
[36006]
[122762]
[42084]
[69935]
[50803]
[14309]
[44294]
[78930]
[83868]
[80554]
[283747]
[54926]
[88045]
[86083]
[36567]
[7597]
[2093]
[2137]
[6518]
[129517]
[48178]
[21

[28318]
[170125]
[222150]
[40490]
[119428]
[105571]
[205568]
[177020]
[67274]
[91732]
[125646]
[47658]
[215637]
[136674]
[95319]
[309422]
[63754]
[48824]
[51257]
[54853]
[60785]
[68916]
[163693]
[122334]
[76896]
[135684]
[119759]
[77708]
[150209]
[79898]
[350683]
[167602]
[117177]
[228126]
[169611]
[88361]
[126783]
[88969]
[260710]
[161485]
[99000]
[102340]
[237915]
[171894]
[292158]
[125926]
[122515]
[140460]
[81079]
[98050]
[86237]
[305445]
[74]
[45073]
[253607]
[11297]
[53453]
[10]
[2264]
[30]
[31]
[33]
[34]
[36]
[39]
[63]
[59]
[61]
[65]
[6523]
[38394]
[16403]
[85]
[88]
[92]
[652]
[101]
[102]
[104]
[129]
[131]
[47581]
[141]
[142]
[287574]
[148]
[155]
[355645]
[171]
[191243]
[218803]
[224]
[449]
[232]
[71501]
[273]
[276]
[283]
[168450]
[299]
[311]
[52077]
[152023]
[332]
[242585]
[355]
[271170]
[186936]
[7463]
[412]
[429]
[259671]
[448]
[78906]
[114804]
[56199]
[103635]
[567]
[571]
[572]
[576]
[580]
[916]
[607]
[622]
[639]
[2532]
[645]
[646]
[650]
[28011]
[665]
[3128]
[6305]
[673]
[67

[15589]
[113776]
[680]
[17984]
[100421]
[694]
[705]
[13450]
[712]
[731]
[751]
[759]
[760]
[772]
[33644]
[779]
[6108]
[794]
[808]
[114806]
[5428]
[903]
[52128]
[945]
[17141]
[78795]
[3833]
[1005]
[1008]
[1013]
[293576]
[1027]
[1028]
[1062]
[1093]
[1094]
[38448]
[1117]
[1118]
[127483]
[7799]
[264968]
[11885]
[323103]
[1251]
[55287]
[1287]
[28305]
[1324]
[254693]
[1346]
[1361]
[1370]
[1372]
[176650]
[1439]
[1576]
[1468]
[1496]
[97604]
[1512]
[81884]
[1566]
[1575]
[47731]
[105282]
[1635]
[268249]
[269613]
[92784]
[1717]
[85822]
[38180]
[32286]
[1774]
[5614]
[1855]
[90856]
[1881]
[191614]
[128500]
[1904]
[1928]
[86621]
[10386]
[112883]
[2014]
[111616]
[41702]
[44932]
[6076]
[197751]
[37774]
[55597]
[7077]
[71483]
[185155]
[2269]
[84747]
[85847]
[93245]
[13037]
[267542]
[88341]
[232360]
[315550]
[260588]
[183388]
[169625]
[48243]
[21789]
[6322]
[96558]
[19960]
[37968]
[7597]
[75764]
[192819]
[95326]
[38860]
[234793]
[2655]
[2872]
[3062]
[3318]
[3434]
[3451]
[3457]
[109458]
[3951]
[56545]
[13

...

[297022]
[279145]
[232487]
[259249]
[247796]
[251470]
[156082]
[7971]
[259720]
[207976]
[129108]
[239568]
[107838]
[10672]
[249986]
[84772]
[263334]
[307489]
[249487]
[173480]
[16814]
[252234]
[301894]
[211516]
[99597]
[221622]
[22790]
[231914]
[96831]
[186293]
[143585]
[219378]
[119709]
[122114]
[213281]
[189696]
[76421]
[165463]
[205179]
[156482]
[245080]
[238468]
[179737]
[75446]
[220784]
[123062]
[318135]
[134064]
[272442]
[119586]
[306647]
[247981]
[146770]
[350694]
[71806]
[246253]
[200370]
[189180]
[113674]
[357546]
[236710]
[116067]
[91732]
[335837]
[261446]
[88298]
[208581]
[261488]
[125646]
[106902]
[209768]
[95319]
[66674]
[63754]
[265024]
[171719]
[54052]
[59173]
[68916]
[71819]
[229927]
[113785]
[140171]
[76468]
[150209]
[143728]
[256146]
[79898]
[93185]
[242728]
[261610]
[117177]
[169611]
[88404]
[135078]
[241988]
[246711]
[88969]
[170746]
[196773]
[161485]
[220257]
[156563]
[265116]
[202679]
[265678]
[148501]
[96528]
[283460]
[106318]
[232307]
[168134]
[138774]
[178740]


[867]
[27842]
[875]
[876]
[35225]
[37198]
[901]
[906]
[10938]
[914]
[170838]
[29573]
[107165]
[918]
[52128]
[79537]
[89202]
[926]
[65545]
[927]
[102896]
[77474]
[944]
[945]
[139601]
[88815]
[17141]
[17112]
[21192]
[963]
[78795]
[38334]
[2354]
[976]
[92971]
[270237]
[176075]
[995]
[90575]
[3833]
[1005]
[168462]
[1876]
[1010]
[1009]
[2319]
[170126]
[1013]
[1017]
[1944]
[58278]
[1027]
[1056]
[1062]
[1065]
[1066]
[37084]
[1068]
[1071]
[1072]
[39908]
[85523]
[4966]
[25580]
[1093]
[42484]
[1094]
[110546]
[1112]
[1115]
[2588]
[1117]
[66294]
[1118]
[89349]
[1119]
[4961]
[44478]
[282090]
[40757]
[264968]
[1153]
[1157]
[42075]
[1159]
[1161]
[1163]
[182505]
[95230]
[31613]
[1186]
[32410]
[78709]
[164471]
[1212]
[1213]
[182951]
[74626]
[1262]
[71908]
[75564]
[1276]
[47763]
[49789]
[9024]
[260706]
[1287]
[28305]
[1299]
[1320]
[33215]
[146806]
[22919]
[2631]
[1331]
[71947]
[25628]
[221362]
[1341]
[1344]
[1346]
[57912]
[1351]
[1352]
[54654]
[1370]
[1367]
[249501]
[1371]
[61978]
[1372]
[77384]
[1696]


[54522]
[103021]
[53689]
[402]
[37123]
[7463]
[403]
[408]
[677]
[411]
[412]
[1075]
[429]
[40686]
[831]
[448]
[78906]
[140393]
[108008]
[77999]
[7675]
[225388]
[32767]
[156991]
[259721]
[270794]
[570]
[571]
[576]
[577]
[77723]
[580]
[582]
[593]
[1830]
[599]
[176446]
[607]
[613]
[615]
[620]
[622]
[623]
[43905]
[627]
[628]
[637]
[645]
[646]
[246173]
[650]
[246262]
[656]
[128659]
[45568]
[15589]
[665]
[97912]
[251585]
[667]
[672]
[237806]
[674]
[228568]
[41696]
[248941]
[680]
[17984]
[691]
[99790]
[693]
[695]
[184960]
[705]
[210559]
[228552]
[94047]
[13450]
[722]
[723]
[12830]
[78007]
[731]
[309441]
[751]
[752]
[754]
[759]
[13209]
[289504]
[772]
[33644]
[774]
[776]
[56469]
[778]
[782]
[788]
[6108]
[795]
[58156]
[808]
[238119]
[80348]
[119760]
[114806]
[820]
[254441]
[1192]
[30750]
[837]
[58609]
[5428]
[47748]
[862]
[76710]
[33012]
[875]
[156012]
[260031]
[169720]
[902]
[903]
[906]
[26618]
[47210]
[914]
[52128]
[89202]
[924]
[225343]
[935]
[942]
[36237]
[260912]
[17141]
[14368]
[17112]
[909

[311821]
[103443]
[227005]
[144870]
[33183]
[43421]
[174714]
[41815]
[60330]
[85571]
[35653]
[133139]
[113468]
[328209]
[90824]
[187180]
[88393]
[264994]
[19436]
[50803]
[71880]
[177235]
[106066]
[19960]
[44294]
[254350]
[186279]
[7597]
[241669]
[116968]
[90131]
[2093]
[157930]
[54952]
[206261]
[2137]
[40671]
[6518]
[279145]
[86487]
[2189]
[143991]
[43080]
[164273]
[265065]
[25852]
[89174]
[2266]
[221417]
[91469]
[156202]
[120140]
[211376]
[2392]
[84292]
[2403]
[2426]
[96304]
[2480]
[7520]
[2511]
[191802]
[79380]
[2558]
[2559]
[196520]
[2671]
[181990]
[2751]
[92083]
[60235]
[216911]
[122853]
[78761]
[185213]
[132105]
[9088]
[111538]
[20578]
[109026]
[51389]
[245259]
[87638]
[77401]
[87423]
[249019]
[271487]
[3434]
[3457]
[71735]
[251001]
[3734]
[97319]
[87679]
[313935]
[170316]
[95930]
[86705]
[104648]
[235211]
[3916]
[3951]
[3968]
[51723]
[56545]
[4109]
[4161]
[56266]
[101102]
[94714]
[228608]
[33127]
[150588]
[4630]
[46414]
[47695]
[47782]
[309859]
[4773]
[232487]
[5057]
[30420]
[154

KeyboardInterrupt: 

In [126]:
for doc in tagged_docs:
    if doc.tags == [284513]:
        print(doc.tags)
#     print (doc.tags)

In [117]:
# There are the same number of beers in each
len(lookup_dict) == len(cleaned_reviews_df.groupby('id').count())

True

In [118]:
lookup_dict[165209]

KeyError: 165209

In [119]:
cleaned_reviews_df.loc[cleaned_reviews_df.id == 165209]

,id,text


In [127]:
beers = pd.read_csv('beers-breweries-and-beer-reviews/beers.csv')

In [142]:
beers.loc[beers.id == 284513]

,id,name,brewery_id,state,country,style,availability,abv,notes,retired
111948,284513,Lunar Mansions,27273,PA,US,Belgian Saison,Rotating,5.7,Saison and the first brewed with this year's f...,f


In [143]:
cleaned_reviews_df.loc[cleaned_reviews_df.id == 284513]

,id,text
